## Web Scraping from wikipedia

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Getting the table from wikipedia 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url, header = 0)

In [3]:
dataframe_raw = dfs[0]


In [ ]:
dataframe_raw

In [4]:
#Replace neighborhoods without a name assigned with it's bororugh name
dataframe_raw.Neighborhood.replace({"NaN":["Borough"]}, inplace=True)

In [5]:
#remove rows with empty Borough
dataframe_raw.dropna(inplace=True)

In [ ]:
dataframe_raw

In [6]:
#Replacing / with ,
dataframe_raw['Neighborhood'] = dataframe_raw['Neighborhood'].str.replace('/',',')

In [7]:
#Sorting the dataframe 
df1 = dataframe_raw.sort_values(['Postal code'])

In [8]:
df1.shape

(103, 3)

## Adding Latitude and Longitude

The CSV file containing the longitude and latitutde data is uploaded to jupyter notebook and imported using pandas

In [9]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_d3d67f40c02443368559cc11e650b690 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='iA-FXz61Q7OWYxH19XfNe50gbQYLbmvhfLGIons8i7Nx',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d3d67f40c02443368559cc11e650b690.get_object(Bucket='capstoneproject-donotdelete-pr-tnn3xfgfq3vs8m',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_ll = pd.read_csv(body)
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df_ll = df_ll.sort_values(['Postal Code'])

In [11]:
df_ll.shape

(103, 3)

In [12]:
#Combining the two dataframes
df_final = df1.merge(df_ll, how='inner', left_on='Postal code', right_on='Postal Code')

In [13]:
df_final.shape

(103, 6)

In [14]:
df_final.head(10)

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",M1N,43.692657,-79.264848


## Neighborhood Exploring and Clustering

In [15]:
#defining Foursquare credentials
CLIENT_ID = 'VOVAKA2MXQHC5LUGHLZJFEUYWSHCCXPOCCQ5BMMFQVZFIGZO' 
CLIENT_SECRET = '4CLILCWQKA2BEZXI2QTSL3DPA3T2XOWMM3BW2J5LG0S05J2C' 
VERSION = '20180605' 

In [16]:
#function to explore all the neighborhoods in toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
import requests
#Getting popular venues in neighbirhoods of Toronto
Toronto_venues = getNearbyVenues(names=df_final['Neighborhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )

Malvern , Rouge
Rouge Hill , Port Union , Highland Creek
Guildwood , Morningside , West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park , Ionview , East Birchmount Park
Golden Mile , Clairlea , Oakridge
Cliffside , Cliffcrest , Scarborough Village West
Birch Cliff , Cliffside West
Dorset Park , Wexford Heights , Scarborough Town Centre
Wexford , Maryvale
Agincourt
Clarks Corners , Tam O'Shanter , Sullivan
Milliken , Agincourt North , Steeles East , L'Amoreaux East
Steeles West , L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview , Henry Farm , Oriole
Bayview Village
York Mills , Silver Hills
Willowdale , Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor , Wilson Heights , Downsview North
Northwood Park , York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill , Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West , Riverdale
India Bazaar , The Beaches 

In [18]:
Toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern , Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern , Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
4,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
5,"Guildwood , Morningside , West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
6,"Guildwood , Morningside , West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
7,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
8,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
9,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center


In [19]:
Toronto_venues.shape

(1349, 7)

In [20]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood , Long Branch",10,10,10,10,10,10
"Bathurst Manor , Wilson Heights , Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",24,24,24,24,24,24
Berczy Park,30,30,30,30,30,30
"Birch Cliff , Cliffside West",4,4,4,4,4,4
"Brockton , Parkdale Village , Exhibition Place",23,23,23,23,23,23
Business reply mail Processing Centre,18,18,18,18,18,18


### Analysis of Neighborhoods

In [21]:
#one hot encoding
one_hot_df = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
one_hot_df['Neighborhood'] = Toronto_venues['Neighborhood']

#Moving the Neighborhood Column to the front
cols = list(one_hot_df)
cols.insert(0, cols.pop(cols.index('Neighborhood')))
one_hot_df = one_hot_df.ix[:, cols]

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [22]:
one_hot_df.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#group the dataset by neighborhood and take the mean of occurance of each venue category
grouped_df = one_hot_df.groupby('Neighborhood').mean().reset_index()

In [24]:
grouped_df.head(10)

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,"Alderwood , Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,"Bathurst Manor , Wilson Heights , Downsview North",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,"Bedford Park , Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.033333,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,"Birch Cliff , Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,"Brockton , Parkdale Village , Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,Business reply mail Processing Centre,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.055556
9,"CN Tower , King and Spadina , Railway Lands , ...",0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [25]:
#function to get top most common calues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
#creating a dataframe to display top 10 venues of each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped_df['Neighborhood']

for ind in np.arange(grouped_df.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped_df.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Lounge,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,"Alderwood , Long Branch",Pizza Place,Pharmacy,Athletics & Sports,Sandwich Place,Pub,Dance Studio,Skating Rink,Coffee Shop,Gym,Drugstore
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Middle Eastern Restaurant,Pizza Place,Ice Cream Shop,Supermarket,Shopping Mall,Restaurant,Fried Chicken Joint,Sushi Restaurant
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
4,"Bedford Park , Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Sushi Restaurant,Juice Bar,Thai Restaurant,Cosmetics Shop,Pharmacy,Pizza Place


### Clustering Neighborhoods

The neighborhoods are divided into 5 clusters using K-means clustering

In [27]:
#K-means clustering
from sklearn.cluster import KMeans

k = 5 #numer of clusters
clustered_neighborhood = grouped_df.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=k, random_state=0).fit(clustered_neighborhood)

kmeans.labels_[0:10] 

array([1, 2, 2, 2, 2, 2, 2, 2, 1, 1], dtype=int32)

In [32]:
#Adding cluster labels to top 10 venues dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#combining neighborhood dataframe and top 10 venues dataframe
df_new = df_final
df_new = df_new.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
df_new.head()

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern , Rouge",M1B,43.806686,-79.194353,1.0,Fast Food Restaurant,Print Shop,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",M1C,43.784535,-79.160497,2.0,Construction & Landscaping,History Museum,Bar,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",M1E,43.763573,-79.188711,1.0,Electronics Store,Breakfast Spot,Medical Center,Bank,Intersection,Mexican Restaurant,Rental Car Location,Yoga Studio,Dim Sum Restaurant,Eastern European Restaurant
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Convenience Store,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,1.0,Fried Chicken Joint,Gas Station,Hakka Restaurant,Bakery,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Bank,Dim Sum Restaurant,Diner


In [33]:
df_new.dropna(inplace=True)
df_new.shape

(98, 17)

## Visulaizing Data

In [34]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

#! pip install --user folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_new['Latitude'], df_new['Longitude'], df_new['Neighborhood'], df_new['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Neighborhoods in Toronto, divided into 5 clusters have been shown in the above map.

### Cluster Details

Cluster 1

In [35]:
df_new.loc[df_new['Cluster Labels'] == 0, df_new.columns[[1] + list(range(5, df_new.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,North York,-79.565963,0.0,Pizza Place,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


In [36]:
df_new.loc[df_new['Cluster Labels'] == 1, df_new.columns[[1] + list(range(5, df_new.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,-79.194353,1.0,Fast Food Restaurant,Print Shop,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
2,Scarborough,-79.188711,1.0,Electronics Store,Breakfast Spot,Medical Center,Bank,Intersection,Mexican Restaurant,Rental Car Location,Yoga Studio,Dim Sum Restaurant,Eastern European Restaurant
4,Scarborough,-79.239476,1.0,Fried Chicken Joint,Gas Station,Hakka Restaurant,Bakery,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Bank,Dim Sum Restaurant,Diner
5,Scarborough,-79.239476,1.0,Playground,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
7,Scarborough,-79.284577,1.0,Bakery,Bus Line,Soccer Field,Ice Cream Shop,Park,Metro Station,Intersection,Bus Station,Ethiopian Restaurant,Electronics Store
8,Scarborough,-79.239476,1.0,Motel,American Restaurant,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
10,Scarborough,-79.273304,1.0,Indian Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Chinese Restaurant,Brewery,Pet Store,Donut Shop,Drugstore,Department Store,Distribution Center
11,Scarborough,-79.295849,1.0,Accessories Store,Middle Eastern Restaurant,Breakfast Spot,Shopping Mall,Sandwich Place,Bakery,Auto Garage,Smoke Shop,Discount Store,Electronics Store
12,Scarborough,-79.262029,1.0,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Lounge,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
17,North York,-79.363452,1.0,Athletics & Sports,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop


Cluster 3

In [37]:
df_new.loc[df_new['Cluster Labels'] == 2, df_new.columns[[1] + list(range(5, df_new.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,-79.160497,2.0,Construction & Landscaping,History Museum,Bar,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
3,Scarborough,-79.216917,2.0,Coffee Shop,Korean Restaurant,Convenience Store,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
6,Scarborough,-79.262029,2.0,Discount Store,Department Store,Coffee Shop,Hobby Shop,Train Station,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
9,Scarborough,-79.264848,2.0,College Stadium,Skating Rink,General Entertainment,Café,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
13,Scarborough,-79.304302,2.0,Pharmacy,Pizza Place,Bank,Shopping Mall,Noodle House,Coffee Shop,Intersection,Gas Station,Italian Restaurant,Fast Food Restaurant
15,Scarborough,-79.318389,2.0,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Cosmetics Shop,Bank,Supermarket,Electronics Store,Thrift / Vintage Store,Sandwich Place,Bubble Tea Shop
18,North York,-79.346556,2.0,Coffee Shop,Clothing Store,Bank,Restaurant,Toy / Game Store,Bakery,Department Store,Food Court,Tea Room,Burger Joint
19,North York,-79.385975,2.0,Café,Bank,Chinese Restaurant,Japanese Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
22,North York,-79.408493,2.0,Coffee Shop,Ramen Restaurant,Sandwich Place,Pizza Place,Café,Japanese Restaurant,Electronics Store,Bank,Steakhouse,Butcher
24,North York,-79.442259,2.0,Coffee Shop,Ramen Restaurant,Sandwich Place,Pizza Place,Café,Japanese Restaurant,Electronics Store,Bank,Steakhouse,Butcher


Cluster 4

In [38]:
df_new.loc[df_new['Cluster Labels'] == 3, df_new.columns[[1] + list(range(5, df_new.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,-79.284577,3.0,Park,Playground,Coffee Shop,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
23,North York,-79.400049,3.0,Park,Convenience Store,Bar,Bank,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
25,North York,-79.329656,3.0,Park,Food & Drink Shop,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
40,East York,-79.338106,3.0,Park,Coffee Shop,Convenience Store,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
48,Central Toronto,-79.383160,3.0,Park,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
50,Downtown Toronto,-79.377529,3.0,Park,Playground,Trail,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
74,York,-79.453512,3.0,Park,Spa,Women's Store,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
98,York,-79.518188,3.0,Park,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


Cluster 5

In [39]:
df_new.loc[df_new['Cluster Labels'] == 4, df_new.columns[[1] + list(range(5, df_new.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,-79.498509,4.0,Baseball Field,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
97,North York,-79.532242,4.0,Paper / Office Supplies Store,Baseball Field,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
